In [2]:
!pip install nltk gensim


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [45]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


def predict_NuExtract(model, tokenizer, text, schema, example=["","",""]):
    schema = json.dumps(json.loads(schema), indent=4)
    input_llm =  "<|input|>\n### Template:\n" +  schema + "\n"
    for i in example:
      if i != "":
          input_llm += "### Example:\n"+ json.dumps(json.loads(i), indent=4)+"\n"
    
    input_llm +=  "### Text:\n"+text +"\n<|output|>\n"
    input_ids = tokenizer(input_llm, return_tensors="pt", truncation=True, max_length=4000).to("cuda")

    output = tokenizer.decode(model.generate(**input_ids)[0], skip_special_tokens=True)
    return output.split("<|output|>")[1].split("<|end-output|>")[0]


model = AutoModelForCausalLM.from_pretrained("numind/NuExtract", trust_remote_code=True, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("numind/NuExtract", trust_remote_code=True)

model.to("cuda")

model.eval()

text = """We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and
automated benchmarks. Our models are released under the Apache 2.0 license.
Code: https://github.com/mistralai/mistral-src
Webpage: https://mistral.ai/news/announcing-mistral-7b/"""

schema = """{
    "Model": {
        "Name": "",
        "Number of parameters": "",
        "Number of token": "",
        "Architecture": []
    },
    "Usage": {
        "Use case": [],
        "Licence": ""
    }
}"""

prediction = predict_NuExtract(model, tokenizer, text, schema, example=["","",""])
print(prediction)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does no

OSError: [Errno 28] No space left on device

In [3]:
import pandas as pd
import time
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords
from groq import Groq
import os
from dotenv import load_dotenv
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# EDA

In [4]:
data = {
    "Company":["Jazz Pharma","Apple"],
    "Short Name":["MSFT","AAPL"],
    "Recommendation":["outperform","buy"],
    "Target Price":["170","180"],
    "Date":["2023-01-15","2023-01-15"],
    "Commentary":["""Q1 Results: Jazz Pharmaceuticals reported strong Q1 results on Thursday, with
revenue of $247 million vs. $196 million in Q1 of FY 13, representing 26% YoY growth.
The GAAP Net Loss was $93 million vs. GAAP Net Income of $43 million in Q1 of FY
13, but $127 million of that was due to an upfront license fee and milestone payment
for JZP-110. Adjusted Net Income was $101 million vs. $84 million in Q1 of FY 13.
These figures were slightly ahead of Goldman Stanley estimates of $240 million in
revenue and $95 million in Adjusted Net Income.
■ FY 14 Guidance: The Company reaffirmed its FY 14 guidance across the board, with
revenue of $1.10 billion – $1.16 billion, Xyrem sales of $755 – $755 million, Erwinaze
at $185 – $200 million, Defitelio at $42 – $52 million, and Adjusted Net Income of
$496 – $520 million. While we believe the market has already priced in these
expectations, we continue to see Jazz as an undervalued, high-growth story going
forward, and we believe that its longer-term revenue, EPS, and EBITDA are likely to
exceed consensus estimates in FY 15, FY 16, and beyond.
■ Catalysts: 1) Possible price increases for Xyrem – Given the historical price increases
and the price ranges for comparable orphan drugs, we believe the company is likely
to announce another round of price increases at the end of FY 14 or early FY 15, and
that there is significant room to grow pricing beyond the current levels. 2) Launch of
new marketing campaigns for Xyrem – Jazz management is in the process of
launching awareness campaigns for narcolepsy patients in key geographies, and has
already reported 11,400 Xyrem patients in Q1, above our FY 14 estimate of ~11,300.
3) Settlement of Roxane lawsuit – We believe this will be decided in Jazz’s favor,
resulting in a delayed entrance for Xyrem generics. Current market expectations
point to generics in FY 19 or FY 20, but we believe FY 21 is more likely (with peak
sales of ~$3.0 billion in FY 20).
■ Our $170.00 target price is based on an FY 14 EV / EBITDA multiple of 20.7x and an FY 15
EV / EBITDA multiple of 15.3x, vs. median peer company multiples of 21.8x and 15.3x,
respectively. Given Jazz’s higher revenue growth, margins, and EBITDA growth, we
believe this is still quite conservative. A DCF analysis with our long-term FCF projections,
a discount rate of 8.07%, and a Terminal FCF growth rate of 0.3% also produces an
implied share price of $168.71.""",
"""Apple expects its revenue in the next quarter ending in December 2024 to increase by low- to mid-single digits year over year. The company looks for services revenue to grow at a similar rate as the nearly 13% growth in fiscal year 2024.
Chief financial officer Luca Maestri was asked by analysts a couple of times in the fourth-quarterearnings callabout iPhone sales in the December quarter. He refused to answer.
Cook was asked about the demand for new iPhones in the coming quarters. He responded: "We're not projecting beyond the current quarter obviously. We just don't do that."
He would only go as far as to say that the company believes Apple Intelligence is "a compelling upgrade reason." He also said that the adoption rate of iOS 18.1 was twice as much as that of iOS 17.1 during the first three days of its launch. Cook added, "So, that clearly shows a level of interest out there."
"""]
}

df = pd.DataFrame(data)


In [5]:
#Assign ID to every text
import numpy as np

df["Text ID"] = np.arange(1, len(df) + 1)
df.set_index("Text ID")

,Company,Short Name,Recommendation,Target Price,Date,Commentary
Text ID,,,,,,
1,Jazz Pharma,MSFT,outperform,170,2023-01-15,Q1 Results: Jazz Pharmaceuticals reported stro...
2,Apple,AAPL,buy,180,2023-01-15,Apple expects its revenue in the next quarter ...


In [6]:
df.head()

,Company,Short Name,Recommendation,Target Price,Date,Commentary,Text ID
0,Jazz Pharma,MSFT,outperform,170,2023-01-15,Q1 Results: Jazz Pharmaceuticals reported stro...,1
1,Apple,AAPL,buy,180,2023-01-15,Apple expects its revenue in the next quarter ...,2


In [7]:
df.value_counts("Recommendation")

Recommendation
buy           1
outperform    1
Name: count, dtype: int64

In [8]:
df.value_counts("Company")

Company
Apple          1
Jazz Pharma    1
Name: count, dtype: int64

In [9]:
df.groupby(["Short Name"]).mean("Target Price")

,Text ID
Short Name,
AAPL,2.0
MSFT,1.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Company         2 non-null      object
 1   Short Name      2 non-null      object
 2   Recommendation  2 non-null      object
 3   Target Price    2 non-null      object
 4   Date            2 non-null      object
 5   Commentary      2 non-null      object
 6   Text ID         2 non-null      int64 
dtypes: int64(1), object(6)
memory usage: 244.0+ bytes


# Removing Stopwords

In [11]:
df["Commentary"] = df["Commentary"].apply(strip_multiple_whitespaces)

In [12]:
# Show list of stopwords
nltk_stopwords = set(stopwords.words('english'))
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some',

In [13]:
# Remove negations from the list
to_remove=['ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'don', "don't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [14]:
for word in to_remove:
    nltk_stopwords.remove(word)

In [15]:
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'd', 'did', 'do', 'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'she', "she's", 'should', "should've", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'who

In [16]:
# Correct apply version
df["Commentary"] = df["Commentary"].apply(lambda comment: remove_stopwords(comment, stopwords=nltk_stopwords))

In [17]:
print(df)

       Company Short Name Recommendation Target Price        Date  \
0  Jazz Pharma       MSFT     outperform          170  2023-01-15   
1        Apple       AAPL            buy          180  2023-01-15   

                                          Commentary  Text ID  
0  Q1 Results: Jazz Pharmaceuticals reported stro...        1  
1  Apple expects revenue next quarter ending Dece...        2  


# Add dates for 3 / 6 / 9 / 12 months

In [18]:
print(df["Commentary"][0])

Q1 Results: Jazz Pharmaceuticals reported strong Q1 results Thursday, revenue $247 million vs. $196 million Q1 FY 13, representing 26% YoY growth. The GAAP Net Loss $93 million vs. GAAP Net Income $43 million Q1 FY 13, $127 million due upfront license fee milestone payment JZP-110. Adjusted Net Income $101 million vs. $84 million Q1 FY 13. These figures slightly ahead Goldman Stanley estimates $240 million revenue $95 million Adjusted Net Income. ■ FY 14 Guidance: The Company reaffirmed FY 14 guidance across board, revenue $1.10 billion – $1.16 billion, Xyrem sales $755 – $755 million, Erwinaze $185 – $200 million, Defitelio $42 – $52 million, Adjusted Net Income $496 – $520 million. While believe market already priced expectations, continue see Jazz undervalued, high-growth story going forward, believe longer-term revenue, EPS, EBITDA likely exceed consensus estimates FY 15, FY 16, beyond. ■ Catalysts: 1) Possible price increases Xyrem – Given historical price increases price ranges c

In [19]:
# transform data column to actual date
df["Date"] = pd.to_datetime(df["Date"],format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Company         2 non-null      object        
 1   Short Name      2 non-null      object        
 2   Recommendation  2 non-null      object        
 3   Target Price    2 non-null      object        
 4   Date            2 non-null      datetime64[ns]
 5   Commentary      2 non-null      object        
 6   Text ID         2 non-null      int64         
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 244.0+ bytes


In [20]:
# Apply DateOffset for each row using 'apply' on the DataFrame
df["Date next Day"] = df["Date"].apply(lambda date: date + pd.DateOffset(days=1))
df["Date after 3 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=3))
df["Date after 6 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=6))
df["Date after 9 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=9))
df["Date after 12 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=12))

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Company               2 non-null      object        
 1   Short Name            2 non-null      object        
 2   Recommendation        2 non-null      object        
 3   Target Price          2 non-null      object        
 4   Date                  2 non-null      datetime64[ns]
 5   Commentary            2 non-null      object        
 6   Text ID               2 non-null      int64         
 7   Date next Day         2 non-null      datetime64[ns]
 8   Date after 3 Months   2 non-null      datetime64[ns]
 9   Date after 6 Months   2 non-null      datetime64[ns]
 10  Date after 9 Months   2 non-null      datetime64[ns]
 11  Date after 12 Months  2 non-null      datetime64[ns]
dtypes: datetime64[ns](6), int64(1), object(5)
memory usage: 324.0+ bytes


# API Connection

In [22]:
"""import yfinance as yf
from datetime import timedelta
import pandas as pd
import numpy as np  # Import numpy to use np.abs


#this function might not be necessary since it makes more sense to look at the whole time frame to see if a target price was reached or not!

def get_stock_price_nearest(ticker, date):
    try:
        # Ensure the date is in Pandas Timestamp format
        target_date = pd.to_datetime(date)

        # Define a range around the target date to fetch data
        start_date = target_date - timedelta(days=5)
        end_date = target_date + timedelta(days=5)

        # Download data within this range
        stock_data = yf.download(ticker, start=start_date, end=end_date)

        if not stock_data.empty:
            # Check if data exists for the exact date
            if target_date in stock_data.index:
                return stock_data.loc[target_date]['Close']
            else:
                # Manually find the closest available trading day using numpy.abs()
                stock_data['diff'] = np.abs(stock_data.index - target_date)
                closest_date = stock_data['diff'].idxmin()  # Get the index of the closest date
                closest_price = stock_data.loc[closest_date]['Close']
                return closest_price
        else:
            return None
    
    except Exception as e:
        return f"An unexpected error occurred for {ticker}: {str(e)}"

#def get_max_stock_price_for_timeframe(short_name,time_frame):
    """

'import yfinance as yf\nfrom datetime import timedelta\nimport pandas as pd\nimport numpy as np  # Import numpy to use np.abs\n\n\n#this function might not be necessary since it makes more sense to look at the whole time frame to see if a target price was reached or not!\n\ndef get_stock_price_nearest(ticker, date):\n    try:\n        # Ensure the date is in Pandas Timestamp format\n        target_date = pd.to_datetime(date)\n\n        # Define a range around the target date to fetch data\n        start_date = target_date - timedelta(days=5)\n        end_date = target_date + timedelta(days=5)\n\n        # Download data within this range\n        stock_data = yf.download(ticker, start=start_date, end=end_date)\n\n        if not stock_data.empty:\n            # Check if data exists for the exact date\n            if target_date in stock_data.index:\n                return stock_data.loc[target_date][\'Close\']\n            else:\n                # Manually find the closest available trad

In [23]:
import pandas as pd

# Load and ensure 'Date' is in datetime format
df_performance_data = pd.read_csv("../data/performance_data.csv")

# Convert 'Date' column to datetime and set it as index (with timezone awareness)
df_performance_data['Date'] = pd.to_datetime(df_performance_data['Date'], utc=True)
df_performance_data = df_performance_data.set_index('Date')

# Function to get the maximum stock price within a date range or the nearest available date
def get_stock_prices(ticker, start_date, end_date=None):
    try:
        # Ensure start_date and end_date are in the same timezone (UTC)
        start_date = pd.to_datetime(start_date, utc=True)
        if end_date is not None:
            end_date = pd.to_datetime(end_date, utc=True)
        else:
            end_date = start_date  # If no end_date, use start_date for a single day

        # Filter the data for the ticker and date range
        filtered_data = df_performance_data.loc[
            (df_performance_data.index >= start_date) & 
            (df_performance_data.index <= end_date), ticker]

        if not filtered_data.empty:
            # Return the maximum price within the filtered date range
            max_price = filtered_data.max()
            min_price = filtered_data.min()
            return float(max_price),float(min_price)
        else:
            # If no data available in the range, find the next available date using asof()
            next_available_data = df_performance_data[ticker].asof(start_date)
            if next_available_data is not None:
                return float(next_available_data)
            else:
                return float('nan'),float('nan')  # Return NaN for missing data
    
    except KeyError:
        return float('nan'),float('nan')  # Return NaN for missing data
    except Exception as e:
        return float('nan'),float('nan')  # Return NaN for missing data

# Example function call
print(get_stock_prices(df["Short Name"][0], df["Date"][0], df["Date next Day"][0]))


235.83848571777344


In [24]:
print(get_stock_prices(df["Short Name"][0], df["Date"][0], df["Date after 3 Months"][0]))

(288.18603515625, 228.6419677734375)


In [25]:
def calculate_prices(row):
    short_name = row['Short Name']
    row['Start Price'] = get_stock_prices(short_name, row['Date'])
    row['One Day after'] = get_stock_prices(short_name,row['Date next Day'])
    row['Max Price after 3 Months'],row["Min Price after 3 Months"] = get_stock_prices(short_name, row['Date'],row['Date after 3 Months'])
    row['Max Price after 6 Months'],row['Min Price after 6 Months'] = get_stock_prices(short_name,row['Date after 3 Months'], row['Date after 6 Months'])
    row['Max Price after 9 Months'],row['Min Price after 9 Months'] = get_stock_prices(short_name, row['Date after 6 Months'], row['Date after 9 Months'])
    #row['Price after 12 Months'] = get_stock_prices(short_name, row['Date after 12 Months'])
    return row
# Apply the function to each row
df = df.apply(calculate_prices, axis=1)

In [26]:
df

,Company,Short Name,Recommendation,Target Price,Date,Commentary,Text ID,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months,Start Price,One Day after,Max Price after 3 Months,Min Price after 3 Months,Max Price after 6 Months,Min Price after 6 Months,Max Price after 9 Months,Min Price after 9 Months
0,Jazz Pharma,MSFT,outperform,170,2023-01-15,Q1 Results: Jazz Pharmaceuticals reported stro...,1,2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15,235.838486,235.838486,288.186035,228.641968,344.776672,272.195526,356.057892,309.814514
1,Apple,AAPL,buy,180,2023-01-15,Apple expects revenue next quarter ending Dece...,2,2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15,133.504333,133.504333,164.873001,133.950150,192.722244,162.481842,195.186279,169.562317


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Company                   2 non-null      object        
 1   Short Name                2 non-null      object        
 2   Recommendation            2 non-null      object        
 3   Target Price              2 non-null      object        
 4   Date                      2 non-null      datetime64[ns]
 5   Commentary                2 non-null      object        
 6   Text ID                   2 non-null      int64         
 7   Date next Day             2 non-null      datetime64[ns]
 8   Date after 3 Months       2 non-null      datetime64[ns]
 9   Date after 6 Months       2 non-null      datetime64[ns]
 10  Date after 9 Months       2 non-null      datetime64[ns]
 11  Date after 12 Months      2 non-null      datetime64[ns]
 12  Start Price               

# Calculate performance of Target Prices

In [28]:
#If the target Price is below the current price, we need to check if it was below the target price
def classifcy_performance(df):
    if df["Target Price"] > df["Start Price"]:
        df["tp_reached_after_3_months"] = df["Target Price"] <= df["Max Price after 3 Months"]
        df["tp_reached_after_6_months"] = df["Target Price"] <= df["MaxPrice after 6 Months"]
        df["tp_reached_after_9_months"] = df["Target Price"] <= df["Max Price after 9 Months"]
    else:
        df["tp_reached_after_3_months"] = df["Target Price"] >= df["Min Price after 3 Months"]
        df["tp_reached_after_6_months"] = df["Target Price"] >= df["Min Price after 6 Months"]
        df["tp_reached_after_9_months"] = df["Target Price"] >= df["Min Price after 9 Months"]

In [29]:
df

,Company,Short Name,Recommendation,Target Price,Date,Commentary,Text ID,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months,Start Price,One Day after,Max Price after 3 Months,Min Price after 3 Months,Max Price after 6 Months,Min Price after 6 Months,Max Price after 9 Months,Min Price after 9 Months
0,Jazz Pharma,MSFT,outperform,170,2023-01-15,Q1 Results: Jazz Pharmaceuticals reported stro...,1,2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15,235.838486,235.838486,288.186035,228.641968,344.776672,272.195526,356.057892,309.814514
1,Apple,AAPL,buy,180,2023-01-15,Apple expects revenue next quarter ending Dece...,2,2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15,133.504333,133.504333,164.873001,133.950150,192.722244,162.481842,195.186279,169.562317


# Split texts in separate sentences

In [30]:
nltk.download('punkt_tab')

# Initialize an empty list to collect sentences with their IDs
sentence_data = []

# Iterate through each text in the DataFrame
for index, row in df.iterrows():
    text_id = row['Text ID']  # Unique Text ID
    sentences = sent_tokenize(row['Commentary'])  # Split the text into sentences

    # Add each sentence to the sentence_data list with a unique Sentence ID within the text
    for i, sentence in enumerate(sentences, start=1):
        sentence_data.append({
            'Text ID': text_id,
            'Sentence ID': i,
            'Sentence': sentence
        })

# Create a new DataFrame from the sentence_data list
df_sentences = pd.DataFrame(sentence_data)

# Display the resulting DataFrame
print(df_sentences)


    Text ID  Sentence ID                                           Sentence
0         1            1  Q1 Results: Jazz Pharmaceuticals reported stro...
1         1            2  The GAAP Net Loss $93 million vs. GAAP Net Inc...
2         1            3  Adjusted Net Income $101 million vs. $84 milli...
3         1            4  These figures slightly ahead Goldman Stanley e...
4         1            5  ■ FY 14 Guidance: The Company reaffirmed FY 14...
5         1            6  While believe market already priced expectatio...
6         1            7  ■ Catalysts: 1) Possible price increases Xyrem...
7         1            8  2) Launch new marketing campaigns Xyrem – Jazz...
8         1            9  3) Settlement Roxane lawsuit – We believe deci...
9         1           10  Current market expectations point generics FY ...
10        1           11  ■ Our $170.00 target price based FY 14 EV / EB...
11        1           12  Given Jazz’s higher revenue growth, margins, E...
12        1 

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Topic Modeling with LDA

# FINBERT

# KeyBERT

# LLM Argument Mining

- Provide List of pre-defined arguments

In [31]:
def find_arguments(text, api_key):
    client = Groq(api_key=api_key)

    instructions = """I will give you a financial text, and here are the instructions:
-Collect the main arguments and assign them to a category.
-Categories must be short, generic, and a maximum of 3 words.
-Check if the argument fits any of the existing categories.
-Create a new category if no match is found.
-Update the existing categories afterwards
-Ignore Target Prices/Recommendation as arguments.
-Don't include brand names in argument
-Classify arguments as positive/negative/neutral based on wording, but only return the raw categories, excluding any words that suggest sentiment (e.g., 'Cost Management' instead of 'Cost Management Growth').
-Return the arguments and sentiment as a dictionary (e.g., {'Category 1': 'positive', 'Category 2': 'negative',...).
-the result should always be in one line and every key and value should be within quotation marks
-Return only the dictionary, no other text."""
    final_prompt = instructions + "\n Text: " +text

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Response must be correct formatted json. No additional text must be provided.",
            },
            {
                "role": "user",
                "content": final_prompt,
            }
        ],
        temperature=0,
        model="llama3-70b-8192",
    )

    return response.choices[0].message.content

In [32]:
# Define a helper function that adds a delay
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

def find_arguments_with_delay(commentary):
    result = find_arguments(commentary,api_key)
    print(result)
    time.sleep(2.5)  # 2.5-second delay
    return result


In [33]:
df_sentences["Provided Arguments"] = df_sentences["Sentence"].apply(find_arguments_with_delay)

{"Revenue Growth": "positive", "Financial Performance": "positive"}
{"Financial Performance": "negative", "License Fees": "negative"}
{"Financial Performance": "positive"}
{"Revenue": "positive", "Financial Performance": "positive"}
{"Revenue Guidance": "neutral", "Sales Performance": "neutral", "Financial Performance": "neutral"}
{"Growth Expectations": "positive", "Revenue Outlook": "positive", "Valuation": "positive"}
{"Pricing Power": "positive", "Growth Potential": "positive"}
{"Marketing Efforts": "positive", "Patient Growth": "positive", "Business Operations": "neutral"}
{"Litigation": "positive", "Product Launch": "neutral"}
{"Market Expectations": "neutral", "Sales Growth": "positive"}
{"Valuation": "neutral"}
{"Revenue Growth": "positive", "Margins": "positive", "EBITDA Growth": "positive"}
{"Discount Rate": "neutral", "FCF Projections": "neutral", "Terminal Growth Rate": "neutral", "Share Price": "neutral"}
{"Revenue Growth": "positive"}
{"Revenue Growth": "positive"}
{"Sale

In [34]:
df_sentences

,Text ID,Sentence ID,Sentence,Provided Arguments
0,1,1,Q1 Results: Jazz Pharmaceuticals reported stro...,"{""Revenue Growth"": ""positive"", ""Financial Perf..."
1,1,2,The GAAP Net Loss $93 million vs. GAAP Net Inc...,"{""Financial Performance"": ""negative"", ""License..."
2,1,3,Adjusted Net Income $101 million vs. $84 milli...,"{""Financial Performance"": ""positive""}"
3,1,4,These figures slightly ahead Goldman Stanley e...,"{""Revenue"": ""positive"", ""Financial Performance..."
4,1,5,■ FY 14 Guidance: The Company reaffirmed FY 14...,"{""Revenue Guidance"": ""neutral"", ""Sales Perform..."
5,1,6,While believe market already priced expectatio...,"{""Growth Expectations"": ""positive"", ""Revenue O..."
6,1,7,■ Catalysts: 1) Possible price increases Xyrem...,"{""Pricing Power"": ""positive"", ""Growth Potentia..."
7,1,8,2) Launch new marketing campaigns Xyrem – Jazz...,"{""Marketing Efforts"": ""positive"", ""Patient Gro..."
8,1,9,3) Settlement Roxane lawsuit – We believe deci...,"{""Litigation"": ""positive"", ""Product Launch"": ""..."
9,1,10,Current market expectations point generics FY ...,"{""Market Expectations"": ""neutral"", ""Sales Grow..."


In [35]:
df_sentences.to_csv("sentences.csv")

In [36]:
#df.to_csv("with_dictionaries.csv")

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Company                   2 non-null      object        
 1   Short Name                2 non-null      object        
 2   Recommendation            2 non-null      object        
 3   Target Price              2 non-null      object        
 4   Date                      2 non-null      datetime64[ns]
 5   Commentary                2 non-null      object        
 6   Text ID                   2 non-null      int64         
 7   Date next Day             2 non-null      datetime64[ns]
 8   Date after 3 Months       2 non-null      datetime64[ns]
 9   Date after 6 Months       2 non-null      datetime64[ns]
 10  Date after 9 Months       2 non-null      datetime64[ns]
 11  Date after 12 Months      2 non-null      datetime64[ns]
 12  Start Price               

In [38]:
import pandas as pd
import json
from collections import Counter, defaultdict

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_sentences["Provided Arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
sentiment_overview_df = pd.DataFrame(sentiment_counts).T
sentiment_overview_df.columns = ["positive", "neutral", "negative"]
sentiment_overview_df.index.name = "Argument"

# Display the overview DataFrame
sentiment_overview_df.reset_index(inplace=True)
sentiment_overview_df.sort_values(by=["positive","negative","neutral"],ascending=False)


,Argument,positive,neutral,negative
0,Revenue Growth,4,0,0
1,Financial Performance,3,1,1
8,Valuation,1,1,0
3,Revenue,1,0,0
6,Growth Expectations,1,0,0
7,Revenue Outlook,1,0,0
9,Pricing Power,1,0,0
10,Growth Potential,1,0,0
11,Marketing Efforts,1,0,0
12,Patient Growth,1,0,0


# Check amount of intensifiers

In [39]:

# Basic Intensifiers
basic_intensifiers = [
    "very", "really", "quite", "so", "too", "totally", "absolutely",
    "completely", "utterly", "entirely", "truly", "fully", "genuinely", "surely"
]

# Strong Intensifiers (includes previous stronger, superlative, and "all/full" intensifiers)
strong_intensifiers = [
    "extremely", "incredibly", "highly", "immensely", "vastly", "remarkably", 
    "tremendously", "phenomenally", "exceptionally", "noticeably", "supremely", 
    "powerfully", "acutely", "enormously", "greatly", "immeasurably", "infinitely",
    "wildly", "lavishly", "ludicrously", "fiercely", "ferociously", "intensely",
    "extraordinarily", "overwhelmingly", "staggeringly", "excessively", "insanely", 
    "unbelievably", "inconceivably", "ridiculously", "crazily", "all-out", 
    "all-around", "full-on", "fully", "flat-out"
]

# Formal/Emotive Intensifiers (combines emotive and formal language for a more academic or dramatic tone)
formal_emotive_intensifiers = [
    "profoundly", "considerably", "significantly", "markedly", "appreciably", 
    "notably", "exceedingly", "far", "deeply", "awfully", "horribly", "dreadfully", 
    "frightfully", "painfully", "absurdly", "scandalously", "outrageously", 
    "shockingly", "horrendously", "terribly", "hideously", "grossly"
]


In [40]:
import re

# Function to count intensifiers in a single text
def count_intensifiers(text, intensifiers):
    # Convert to lowercase to make it case-insensitive
    words = re.findall(r'\b\w+\b', text.lower())
    return sum(words.count(word) for word in intensifiers)

In [41]:
# Apply the function to the "Sentence" column of the DataFrame
df_sentences["Basic Intensifier"] = df_sentences["Sentence"].apply(count_intensifiers, intensifiers=basic_intensifiers)
df_sentences["Strong Intensifier"] = df_sentences["Sentence"].apply(count_intensifiers, intensifiers=strong_intensifiers)
df_sentences["Formal Intensifier"] = df_sentences["Sentence"].apply(count_intensifiers, intensifiers=formal_emotive_intensifiers)

In [42]:
df_sentences

,Text ID,Sentence ID,Sentence,Provided Arguments,Basic Intensifier,Strong Intensifier,Formal Intensifier
0,1,1,Q1 Results: Jazz Pharmaceuticals reported stro...,"{""Revenue Growth"": ""positive"", ""Financial Perf...",0,0,0
1,1,2,The GAAP Net Loss $93 million vs. GAAP Net Inc...,"{""Financial Performance"": ""negative"", ""License...",0,0,0
2,1,3,Adjusted Net Income $101 million vs. $84 milli...,"{""Financial Performance"": ""positive""}",0,0,0
3,1,4,These figures slightly ahead Goldman Stanley e...,"{""Revenue"": ""positive"", ""Financial Performance...",0,0,0
4,1,5,■ FY 14 Guidance: The Company reaffirmed FY 14...,"{""Revenue Guidance"": ""neutral"", ""Sales Perform...",0,0,0
5,1,6,While believe market already priced expectatio...,"{""Growth Expectations"": ""positive"", ""Revenue O...",0,0,0
6,1,7,■ Catalysts: 1) Possible price increases Xyrem...,"{""Pricing Power"": ""positive"", ""Growth Potentia...",0,0,0
7,1,8,2) Launch new marketing campaigns Xyrem – Jazz...,"{""Marketing Efforts"": ""positive"", ""Patient Gro...",0,0,0
8,1,9,3) Settlement Roxane lawsuit – We believe deci...,"{""Litigation"": ""positive"", ""Product Launch"": ""...",0,0,0
9,1,10,Current market expectations point generics FY ...,"{""Market Expectations"": ""neutral"", ""Sales Grow...",0,0,0


In [43]:
df["Commentary"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2 entries, 0 to 1
Series name: Commentary
Non-Null Count  Dtype 
--------------  ----- 
2 non-null      object
dtypes: object(1)
memory usage: 148.0+ bytes


In [44]:
print(df[df["Formal Intensifier"] > 1])

KeyError: 'Formal Intensifier'